# MBA em Ciência de Dados
**Técnicas Avançadas de Captura e Tratamento de Dados**

---


Carolinne dos Santos Pinheiro


---




In [ ]:
from PIL import Image
from pdf2image import convert_from_path
from skimage import feature
from os import listdir

import pytesseract as ocr
import nltk
import re
import os
import numpy
import pandas
import imageio
import matplotlib.pyplot as plt


In [ ]:
# Exercício 1

pdf = 'modcovid.pdf'
png = 'modcovid.png'
txt = 'modcovid.txt'

obj_pdf = convert_from_path(pdf)
obj_pdf[0].save(png, 'PNG')

ocr_txt = ocr.image_to_string(Image.open(png), lang='por') # testar lang='pt'

ObjFileW = open(txt, 'w')
ObjFileW.write(ocr_txt)
ObjFileW.close()

In [ ]:
# Exercício 2

ObjFileR = open(txt, 'r')
FileContent = ObjFileR.read()
ObjFileR.close()

def FixDocStrings(v_String):
    StringArr = nltk.word_tokenize(v_String)
    StringArr = [Plv for Plv in StringArr if Plv.isalpha()]
    return StringArr

FixedContent = FixDocStrings(FileContent)

print('O número de palavras com somente um caractere é: %d' % (len([Plv for Plv in FixedContent if len(Plv) == 1])))

In [ ]:
# Exercício 3A

ArquivoCSV = 'artists_mba21.csv'
PdCSV = pandas.read_csv(ArquivoCSV)
PdCSV['genre'] = PdCSV['genre'].str.split(',')

PdCSVArr = []
for Row in PdCSV[['name', 'genre']].iterrows():
    for Genre in Row[1]['genre']:
        PdCSVArr.append({'name': Row[1]['name'], 'genre': Genre})

Pd3A = pandas.DataFrame.from_dict(PdCSVArr)
Pd3A.groupby(['genre']).count().hist(); # verificar %colors


In [ ]:
# Exercício 3B

for Row in PdCSV[['paintings']].iterrows():
    if (re.findall('[^0-9]', Row[1]['paintings'])):
        print(Row[0], Row[1]['paintings'])

In [ ]:
def FixPaintings(v_Num):
    if (v_Num == 'three'):
        return '3'
    else:
        return ''.join(re.findall('[0-9]+', v_Num))

PdCSV['paintings'] = PdCSV['paintings'].apply(FixPaintings).astype('int32')
PdCSV[['paintings']].describe()

In [1]:
# Exercício 4

def rescale_image_colors(img, n_colors=32):
    img = numpy.array(img, dtype=numpy.float64, copy=False)
    if (len(img.shape) > 2):
        img = img[:,:,0]*0.2126 + img[:,:,1]*0.7152 + img[:,:,2]*0.0772

    img = img/255.0
    img = (img*(n_colors-1)).astype(numpy.uint8)
    return img

def euclidean_distance(ArrA, ArrB):
    return numpy.sqrt(numpy.sum((ArrA - ArrB)**2))

In [ ]:
def bic(img, n_colors): # função do monitor

    a_interior = np.zeros(n_colors)
    a_border = np.zeros(n_colors)

    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
            curr = img[i,j]
            
            #bordas da figura
            if i == 0 or i == (img.shape[0] - 1):
                a_border[curr] += 1
                continue
            if j == 0 or j == (img.shape[1] - 1):
                a_border[curr] += 1
                continue
            
            # vizinhos           
            if img[i - 1, j] =! curr:
                a_border[curr] += 1
                continue
            if img[i + 1, j] =! curr:
                a_border[curr] += 1
                continue
            if img[i, j - 1] =! curr:
                a_border[curr] += 1
                continue
            if img[i, j + 1] =! curr:
                a_border[curr] += 1
                continue

            a_interior[curr] += 1
            
    a_interior = a_interior / a_interior.sum()
    a_border = a_border / a_border.sum()
    
    a_bic = np.hstack([a_interior, a_border])
    a_bic = a_bic / a_bic.sum()
    
    return a_bic

In [ ]:
FilesPath = 'paintings21/'

RefImg = ''
Images = []
for FileName in os.listdir(FilesPath):
    FullFileName = os.path.abspath(FilesPath + '/' + FileName)
    if (os.path.isfile(FullFileName)):
        if (FileName == 'query.jpg'):
            RefImg = FullFileName
        else:
            Images.append(FullFileName)
Images.sort()
RefImgBIC = bic(RefImg)

In [ ]:
imgs = {}
for Image in Images:
    BaseName = os.path.basename(Image)
    imgs[BaseName] = euclidean_distance(bic(Image), RefImgBIC)

In [ ]:
img_idx = 161
plt.figure(figsize=(20, 20))
plt.subplot(img_idx); plt.imshow(imageio.imread(RefImg)); plt.title('Referência');

for Similar in sorted(imgs.items(), key=lambda item:item[1])[0:5]:
    img_idx += 1
    plt.subplot(img_idx)
    plt.imshow(imageio.imread(FilesPath + Similar[0]))
    plt.title('Id: %s\nBIC: %.5f' % (Similar[0], Similar[1]))